In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install gradio
!apt-get install timidity
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
timidity is already the newest version (2.13.2-41).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.4 MB/s 
     |████████████████████████████████| 7.6 MB 46.0 MB/s 
     |████████████████████████████████| 182 kB 62.3 MB/s 


In [6]:
import gradio as gr
from pathlib import Path
import subprocess
from transformers import AutoTokenizer
from transformers import TFAutoModelForCausalLM
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import numpy as np

In [7]:
#load poem generation model
model = TFAutoModelForCausalLM.from_pretrained('drive/MyDrive/FIRE_3rd Sem/peom_gn/')
base_model = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at drive/MyDrive/FIRE_3rd Sem/peom_gn/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
#load sentiment analysis
model_v1 = TFAutoModelForSequenceClassification.from_pretrained('drive/MyDrive/FIRE_3rd Sem/sen_analysis/bert')
base_model = "distilbert-base-uncased"
tokenizer_v1 = AutoTokenizer.from_pretrained(base_model)

Some layers from the model checkpoint at drive/MyDrive/FIRE_3rd Sem/sen_analysis/bert were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at drive/MyDrive/FIRE_3rd Sem/sen_analysis/bert and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
#music generation
base_path = "/content/drive/MyDrive/FIRE_3rd Sem/music_gn/"
#path_mid_file -> Replace this with model generated file path
path_mid_file = base_path + "Comic_Relief.mid"
path_wav_file = base_path + "output_comic.wav"
subprocess.call(['timidity', path_mid_file, "-Ow", "-o", path_wav_file])

0

## Music Generation

In [10]:
def inference_music_gen():
  return Path(path_wav_file)

In [13]:
music_gen_interface = gr.Interface(
    inference_music_gen, 
    inputs = None, 
    outputs = gr.outputs.Audio(type="filepath", label="Output")
    )

## Sentiment Analysis

In [14]:
def inference_sentiment_analysis(sen):
  tokenized_v1 = tokenizer_v1([sen], return_tensors="np", padding="longest")
  outputs_v1 = model_v1(tokenized_v1).logits
  classifications_v1 = np.argmax(outputs_v1, axis=1)
  if classifications_v1[0] == 1:
      res = "Positive :)"
  else:
      res = "Negative :("
  return res

In [15]:
sentiment_analysis_interface = gr.Interface(
    fn=inference_sentiment_analysis,
    inputs=gr.Textbox(lines=2, placeholder="Enter a Sentence"),
    outputs="text",
)

## Peom Generation

In [16]:
def inference_poem_gen(start):
  tokenized = tokenizer(start, return_tensors="np")
  outputs = model.generate(**tokenized, max_new_tokens=20)
  res = tokenizer.decode(outputs[0])
  return res.replace("<LINE>", "\n")

In [17]:
poem_gen_interface = gr.Interface(
    fn=inference_poem_gen,
    inputs=gr.Textbox(lines=2, placeholder="Start Here..."),
    outputs="text",
)

## Combine All

In [18]:
title = "Music Generation"
description = "Add Project description"
article = "<p style='text-align: center'><a href='https://github.com/' target='_blank'>Github Repo</a></p>"
#we can add other project related stuff as well

demo = gr.TabbedInterface([music_gen_interface, poem_gen_interface, sentiment_analysis_interface], 
                          ["Music Generation", "Poem Generation", "Sentiment Analysis"])
demo.launch(debug=True, share=True)

/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:717: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:717: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://632e922be0d60ff3.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Exception in thread Thread-22:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/gradio/tunneling.py", line 39, in handler
    chan.send(data)
  File "/usr/local/lib/python3.8/dist-packages/paramiko/channel.py", line 801, in send
    return self._send(s, m)
  File "/usr/local/lib/python3.8/dist-packages/paramiko/channel.py", line 1198, in _send
    raise socket.error("Socket is closed")
OSError: Socket is closed
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Keyboard interruption in main thread... closing server.
